In [9]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# google drive 접근 설정
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
data = pd.read_csv('/content/drive/MyDrive/movies_metadata.csv', low_memory = False)
data.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [ ]:
# 영화 줄거리에 기반한 영화 추천 시스템

In [11]:
# 상위 2만개의 샘플을 data에 저장
data = data.head(20000)

In [12]:
# overview 열에 존재하는 모든 결측값을 전부 카운트하여 출력
print('overview 열의 결측값의 수:',data['overview'].isnull().sum())

overview 열의 결측값의 수: 135


In [13]:
# 결측값을 빈 값으로 대체
data['overview'] = data['overview'].fillna('')
# fillna() : 결측값이 있던 행에 특정값으로 채워넣는 함수
# dropna() : 결측값을 가진 행을 제거

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [16]:
# data의 overview열에 대해 TF-IDF 행렬을 구함
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['overview'])
print('TF-IDF 행렬의 크기(shape) :',tfidf_matrix.shape)

TF-IDF 행렬의 크기(shape) : (20000, 47487)


In [17]:
# 코사인 유사도 계산
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print('코사인 유사도 연산 결과 :', cosine_sim.shape)

코사인 유사도 연산 결과 : (20000, 20000)


In [18]:
title_to_index = dict(zip(data['title'], data.index))

# 영화 제목 Father of the Bride Part II의 인데스를 리턴
idx = title_to_index['Father of the Bride Part II']
print(idx)

4


In [19]:
# 선택한 영화의 제목을 입력하면 코사인 유사도를 통해 가장 overview가 유사한 10개의 
# 영화를 찾아내는 함수

In [44]:
def get_recommendations(title, cosine_sim = cosine_sim):
  # 선택한 영화의 타이틀로부터 해당 영화의 인덱스를 받아온다
  idx = title_to_index[title]

  # 해당 영화와 모든 영화와의 유사도를 가져온다
  sim_score = list(enumerate(cosine_sim[idx]))

  # 유사도에 따라 영화들을 정렬한다
  sim_score = sorted(sim_score, key=lambda x: x[1], reverse=True)

  # 가장 유사한 10개의 영화를 받아온다
  sim_socre = sim_score[1:11]

  # 가장 유사한 10개의 영화의 인덱스를 얻는다
  movie_indices = [idx[0] for idx in sim_socre]

  #가장 유사한 10개 영화의 제목을 리턴한다
  return data['title'].iloc[movie_indices]

In [47]:
# The Dark Knight Rises와 overvies가 유사한 10개의 영화를 찾아낸다
recommend = get_recommendations('The Dark Knight Rises')
recommend

12481                            The Dark Knight
150                               Batman Forever
1328                              Batman Returns
15511                 Batman: Under the Red Hood
585                                       Batman
9230          Batman Beyond: Return of the Joker
18035                           Batman: Year One
19792    Batman: The Dark Knight Returns, Part 1
3095                Batman: Mask of the Phantasm
10122                              Batman Begins
Name: title, dtype: object